In [ ]:
# Import required libraries
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from google.oauth2 import service_account
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from google.oauth2 import service_account
from google.cloud import bigquery
from selenium import webdriver
from bs4 import BeautifulSoup
from io import StringIO
import pandas_gbq as pg
import pandas as pd
import numpy as np
import db_dtypes
import requests
import datetime
import socket
import json
import time
import re

# https://docs.google.com/spreadsheets/d/1FHMiINu9_3KSEuconyMFQWsO6K9MbM9W6xlbXC2QR0Q/edit?gid=0#gid=0

# Define the Google Sheets document ID
sheet_id = '1FHMiINu9_3KSEuconyMFQWsO6K9MbM9W6xlbXC2QR0Q'

# Load the data from the specified Google Sheets document into a pandas DataFrame
xls = pd.ExcelFile(f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=xlsx")
Url_List = pd.read_excel(xls, 'URL Input Sheet ')

# Prepare URL DataFrame
url_df = Url_List
url_df['index'] = url_df.index
url_df['Rand_Num'] = url_df.index.to_series().apply(lambda x: f"?{1000 + x}")  # to clear chache referesh the website 
url_df['New_Url'] = url_df['Url'] + url_df['Rand_Num']
url_lstt = url_df.values

# Initialize DataFrame to collect results and list to collect failed URLs
data_df = pd.DataFrame()
failed_urls = []

# Initialize WebDriver for Chrome
driver_path = r'C:\Users\Maaz Shaikh\chromedriver-win64\chromedriver.exe'
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# Loop through each URL in the list and scrape price data
for i in url_lstt:
    Price = None

    # Handling different sources (e.g., Tatacliq, Amazon, Flipkart, etc.)
    if i[2] == "Tatacliq":
        try:
            driver.get(i[5])
            time.sleep(2)  # Wait for the page to load
            text_field = driver.find_element(By.XPATH, '//*[@id="BPDT"]/div[1]/div[2]/div[1]/div[1]/h3')
            Price = text_field.text
        except:
            try:
                driver.get(i[5])
                time.sleep(2)
                text_field = driver.find_element(By.XPATH, '//*[@id="app"]/div[2]/div/div/div[1]/div[2]/div[1]/div[1]/div[2]')                             
                Price = text_field.text
            except:
                failed_urls.append(i[1])

    elif i[2] == "Amazon":
        try:
            driver.get(i[5])
            time.sleep(2)
            text_field = driver.find_element(By.XPATH, '//span[text()=" Currently unavailable. "]')
            Price = text_field.text
        except:
            try:
                driver.get(i[5])
                time.sleep(2)
                text_field = driver.find_element(By.XPATH, '(//span[@class="a-price-whole"])[1]')
                Price = text_field.text
            except:
                try:
                    driver.get(i[5])
                    time.sleep(2)
                    text_field = driver.find_element(By.XPATH, '//*[@id="corePrice_feature_div"]/div/div/span[1]/span[2]/span[2]')
                    Price = text_field.text
                except:
                    failed_urls.append(i[1])

    elif i[2] == "Flipkart":
        try:
            driver.get(i[5])
            time.sleep(5)  # Longer wait time as Flipkart might take more time to load
            text_field = driver.find_element(By.XPATH, '//*[@id="container"]/div/div[3]/div[1]/div[2]/div[2]/div/div[2]/div[1]/div/div')
            Price = text_field.text
        except:
            try:
                driver.get(i[5])
                time.sleep(2)
                text_field = driver.find_element(By.XPATH, '//div//div[@class="Nx9bqj CxhGGd"]')
                Price = text_field.text
            except:
                failed_urls.append(i[1])

    elif i[2] == "Ajio":
        try:
            driver.get(i[5])
            time.sleep(2)
            text_field = driver.find_element(By.XPATH, '//div//div[@class="prod-sp"]')
            Price = text_field.text
        except:
            try:
                driver.get(i[5])
                time.sleep(2)
                text_field = driver.find_element(By.XPATH, '//*[@id="appContainer"]/div[2]/div/div/div[2]/div/div[3]/div/div[2]/div')
                Price = text_field.text
            except:
                failed_urls.append(i[1])

    elif i[2] == "Myntra":
        try:
            driver.get(i[5])
            time.sleep(2)
            text_field = driver.find_element(By.XPATH, '//*[@id="mountRoot"]/div/div[1]/main/div[2]/div[2]/div[1]/div/p[1]/span[1]')
            Price = text_field.text
        except:
            try:
                driver.get(i[5])
                time.sleep(2)
                text_field = driver.find_element(By.XPATH, '//span[@class="pdp-price"]')
                Price = text_field.text
            except:
                failed_urls.append(i[1])

    else:
        print("Error: Unsupported source")
        continue

    # Create a dictionary for the current entry and append it to the results DataFrame
    dictt = {
        "Sr_No": i[3],
        "Brand": i[0],
        "Url": i[1],
        "Price": Price,
        "Source": i[2]
    }
    
    df = pd.DataFrame(dictt, index=[0])
    data_df = pd.concat([data_df, df], axis=0)

# Close the WebDriver
driver.quit()

# Get the local machine's hostname and IP address
hostname = socket.gethostname()
IPAddr = socket.gethostbyname(hostname)

# Define Google Cloud credentials
key_path = {
    "type": "",
    "project_id": "",
    "private_key_id": "",
    "private_key": "",
    "client_email": "",
    "client_id": "",
    "auth_uri": "",
    "token_uri": "",
    "auth_provider_x509_cert_url": "",
    "client_x509_cert_url": "",
    "universe_domain": ""
}

# Set up Google Cloud BigQuery client
credentials = service_account.Credentials.from_service_account_info(
    key_path,
    scopes=["https://www.googleapis.com/auth/cloud-platform"]
)

client = bigquery.Client(
    credentials=credentials,
    project=credentials.project_id
)

# Convert the DataFrame to string type and upload it to Google BigQuery
data_df = data_df.astype("string")
pg.to_gbq(data_df, 'Dummy_Dataset.image_link_price', project_id="XYZ", if_exists='append', credentials=credentials)